In [2]:
%load_ext autoreload
%autoreload 2
import datetime
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import casadi as cas

##### For viewing the videos in Jupyter Notebook
import io
import base64
from IPython.display import HTML

# from ..</src> import car_plotting
# from .import src.car_plotting
PROJECT_PATH = '/home/nbuckman/Dropbox (MIT)/DRL/2020_01_cooperative_mpc/mpc-multiple-vehicles/'
sys.path.append(PROJECT_PATH)
import src.MPC_Casadi as mpc
import src.car_plotting as cplot

%matplotlib inline

## Get better results when penalizing the final length rather than the velocity at each time step

# Vehicle Dynamics $\frac{d}{dt} \vec{x} = f(\vec{x}, \vec{u})$

#  States

$\vec{x}$ = $[x, y, \phi, \delta, V, s]^T$

$\vec{u}$ = $[\delta^u, v^u]^T$

# Discrete (integrated) dynamics $\vec{x}_{t+1} = F(\vec{x}_{t}, \vec{u}_{t})$

In [3]:
T = 10 #numbr of time horizons
dt = 0.1
N = int(T/dt) #Number of control intervals

# Problem Definition
### Parameterization of Desired Trajectory ($\vec{x}_d = f_d(s)$)

In [4]:
s = cas.MX.sym('s')
xd = s
yd = 0
phid = 0
des_traj = cas.vertcat(xd, yd, phid)
fd = cas.Function('fd',[s],[des_traj],['s'],['des_traj'])

In [5]:
#Globally true information
min_dist = 2 * (2 * .5**2)**.5

# initial_speed = 6.7 
initial_speed = 20 * 0.447 # m/s

# Initial Conditions
x0 = np.array([2*min_dist, 1.2*min_dist, 0, 0, initial_speed, 0]).T
x0_2 = np.array([2*min_dist, 0, .0, 0, initial_speed, 0]).T
x0_amb = np.array([0, 0.0, 0, 0, 1.1 * initial_speed , 0]).T

In [6]:
LANE_WIDTH = min_dist
xd2 = s
yd2 = LANE_WIDTH
phid = 0
des_traj2 = cas.vertcat(xd2, yd2, phid)
fd2 = cas.Function('fd',[s],[des_traj2],['s'],['des_traj2'])

## Warm Start

### Solve it centrally just to warm start the solution

In [7]:
x1_MPC = mpc.MPC(dt)
x2_MPC = mpc.MPC(dt)
amb_MPC = mpc.MPC(dt)

x1_MPC.k_s = -1.0
x2_MPC.k_s = -1.0
amb_MPC.k_s = -1.0

x1_MPC.k_v = 0
x2_MPC.k_v = 0
amb_MPC.k_v = 0



x1_MPC.theta_iamb = np.pi/4
x2_MPC.theta_iamb = np.pi/4


amb_MPC.theta_iamb = 0.0


# amb_MPC.k_u_v = 0.10
# amb_MPC.k_u_change = 1.0
amb_MPC.max_v = 40 * 0.447 # m/s
amb_MPC.max_X_dev = 5.0


x2_MPC.fd = fd
amb_MPC.fd = fd
x1_MPC.fd = fd2


x1_MPC.min_y = -1.1 * LANE_WIDTH
x2_MPC.min_y = -1.1 * LANE_WIDTH
amb_MPC.min_y = -1.1 * LANE_WIDTH

In [8]:
opt =  mpc.OptimizationMPC(x1_MPC, x2_MPC,amb_MPC)
opt.generate_optimization(N, min_dist, fd, T, x0, x0_2, x0_amb, 2)

In [8]:
opt.solve()
x1, u1, x1_des, x2, u2, x2_des, xamb, uamb, xamb_des = opt.get_solution()

In [9]:
optional_suffix = "_s"
subdir_name = ''
folder = "results/" + subdir_name + "/"
os.makedirs(folder)
os.makedirs(folder+"imgs/")
print(folder)

FileExistsError: [Errno 17] File exists: 'results/20200219-112151_s/'

In [10]:
opt.save_state(folder,x1, u1, x1_des, x2, u2, x2_des, xamb, uamb, xamb_des)

'results/20200219-112151_s/'

In [10]:
subdir_name = '20200219-112151_s'
folder = "results/" + subdir_name + "/"
x1, u1, x1_des, x2, u2, x2_des, xamb, uamb, xamb_des = opt.load_state(folder)

In [24]:
t_start = 5
di = 4
t_end = 55
frames = np.arange(t_start, t_end, di)
print(frames)

[ 5  9 13 17 21 25 29 33 37 41 45 49 53]


In [25]:
cplot.plot_cars_singleframe(x1[:, frames], x2[:, frames], xamb[:, frames], folder,x1_des[:, frames], x2_des[:, frames], xamb_des[:, frames], CIRCLES=False, min_dist=-1, SLIDING_WINDOW=False)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [27]:
ymin, ymax = -0.9703618171097358, 2.865653861652571
figwidth_in = 12.0
axlim_maxx = 80.0
axlim_minx = -0
np.ceil(1.1 * figwidth_in * (ymax - ymin) / (axlim_maxx - axlim_minx ))

1.0

In [31]:
CIRCLES = False
if CIRCLES:
    vid_fname = folder + subdir_name + 'circle_central.mp4'
else:
    vid_fname = folder + subdir_name + 'car_central.mp4'   
if os.path.exists(vid_fname):
    os.remove(vid_fname)
cmd = 'ffmpeg -r 16 -f image2 -i {}imgs/%03d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p {}'.format(folder, vid_fname)
os.system(cmd)
print('Saving video to: {}'.format(vid_fname))

Saving video to: results/20200219-103827_s/20200219-103827_scar_central.mp4


In [32]:
video = io.open(vid_fname, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))


## Now switch to penalizing V

##### Let's choose a k_s that has reasonable magnitude

In [33]:
print(opt.solution.value(x1_MPC.s_cost))
print(opt.solution.value(x2_MPC.s_cost))
print(opt.solution.value(amb_MPC.s_cost))

print(opt.solution.value(x1_MPC.v_cost))
print(opt.solution.value(x2_MPC.v_cost))
print(opt.solution.value(amb_MPC.v_cost))

2470.0158345361465
2454.696990550646
2934.318915282058
5056.337284878917
5023.910425573446
6003.52771273942


In [35]:
x1_MPC = mpc.MPC(dt)
x2_MPC = mpc.MPC(dt)
amb_MPC = mpc.MPC(dt)

x1_MPC.k_s = 0
x2_MPC.k_s = 0
amb_MPC.k_s = 0

x1_MPC.k_v = -.5
x2_MPC.k_v = -.5
amb_MPC.k_v = -.5



x1_MPC.theta_iamb = np.pi/4
x2_MPC.theta_iamb = np.pi/4


amb_MPC.theta_iamb = 0.0


# amb_MPC.k_u_v = 0.10
# amb_MPC.k_u_change = 1.0
amb_MPC.max_v = 40 * 0.447 # m/s
amb_MPC.max_X_dev = 5.0


x2_MPC.fd = fd
amb_MPC.fd = fd
x1_MPC.fd = fd2


x1_MPC.min_y = -1.1 * LANE_WIDTH
x2_MPC.min_y = -1.1 * LANE_WIDTH
amb_MPC.min_y = -1.1 * LANE_WIDTH

In [36]:
opt =  mpc.OptimizationMPC(x1_MPC, x2_MPC,amb_MPC)
opt.generate_optimization(N, min_dist, fd, T, x0, x0_2, x0_amb, 2)
opt.solve()

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  24.31ms (243.09us)  24.42ms (244.18us)       100
       nlp_g  |  71.32ms (713.22us)  71.34ms (713.37us)       100
    nlp_grad  |   2.02ms (  2.02ms)   2.02ms (  2.02ms)         1
  nlp_grad_f  |  33.05ms (478.99us)  32.97ms (477.78us)        69
  nlp_hess_l  | 992.32ms ( 10.56ms) 988.99ms ( 10.52ms)        94
   nlp_jac_g  | 493.58ms (  5.04ms) 492.12ms (  5.02ms)        98
       total  |   2.44 s (  2.44 s)   2.43 s (  2.43 s)         1


In [37]:
x1, u1, x1_des, x2, u2, x2_des, xamb, uamb, xamb_des = opt.get_solution()

In [38]:
optional_suffix = "_v"
subdir_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + optional_suffix
folder = "results/" + subdir_name + "/"
os.makedirs(folder)
os.makedirs(folder+"imgs/")
print(folder)

results/20200219-104449_v/


In [39]:
opt.save_state(folder,x1, u1, x1_des, x2, u2, x2_des, xamb, uamb, xamb_des)

'results/20200219-104449_v/'

In [40]:
cplot.plot_cars(x1, x2, xamb, folder,x1_des, x2_des, xamb_des)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [41]:
CIRCLES = False
if CIRCLES:
    vid_fname = folder + subdir_name + 'circle_central.mp4'
else:
    vid_fname = folder + subdir_name + 'car_central.mp4'   
if os.path.exists(vid_fname):
    os.remove(vid_fname)
cmd = 'ffmpeg -r 16 -f image2 -i {}imgs/%03d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p {}'.format(folder, vid_fname)
os.system(cmd)
print('Saving video to: {}'.format(vid_fname))

Saving video to: results/20200219-104449_v/20200219-104449_vcar_central.mp4


In [42]:
video = io.open(vid_fname, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))
